In [23]:
import src

ModuleNotFoundError: No module named 'utilities'

In [19]:
checkpoint = "meta-llama/Llama-3.2-1B"
llamaSkipConfig = LlamaSkipConnectionConfig.from_pretrained(checkpoint)
llamaSkipModel = LlamaSkipConnectionModel.from_pretrained(checkpoint, config=llamaSkipConfig)

loading configuration file config.json from cache at /home/azureuser/.cache/huggingface/hub/models--meta-llama--Llama-3.2-1B/snapshots/221e3535e1ac4840bdf061a12b634139c84e144c/config.json
Model config LlamaSkipConnectionConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_v

In [20]:
llamaSkipConfig.save_pretrained("llama-skip-connection")

Configuration saved in llama-skip-connection/config.json
